In [ ]:
import cv2
import numpy as np
import math

def nothing(x):
    pass

cap = cv2.VideoCapture(0)
window_name = "Arvyax Final Assignment"
cv2.namedWindow(window_name)

cv2.createTrackbar("Hue Min", window_name, 0, 179, nothing)
cv2.createTrackbar("Hue Max", window_name, 20, 179, nothing)
cv2.createTrackbar("Sat Min", window_name, 50, 255, nothing)
cv2.createTrackbar("Sat Max", window_name, 255, 255, nothing)
cv2.createTrackbar("Val Min", window_name, 50, 255, nothing)
cv2.createTrackbar("Val Max", window_name, 255, 255, nothing)

OBJECT_RADIUS = 50
SAFE_DIST = 200
DANGER_DIST = 80

while True:
    success, frame = cap.read()
    if not success:
        break
    
    frame = cv2.flip(frame, 1)
    height, width, _ = frame.shape
    
    center_x = width // 2
    center_y = height // 2

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    h_min = cv2.getTrackbarPos("Hue Min", window_name)
    h_max = cv2.getTrackbarPos("Hue Max", window_name)
    s_min = cv2.getTrackbarPos("Sat Min", window_name)
    s_max = cv2.getTrackbarPos("Sat Max", window_name)
    v_min = cv2.getTrackbarPos("Val Min", window_name)
    v_max = cv2.getTrackbarPos("Val Max", window_name)

    lower = np.array([h_min, s_min, v_min])
    upper = np.array([h_max, s_max, v_max])
    mask = cv2.inRange(hsv, lower, upper)

    kernel = np.ones((5,5), np.uint8)
    mask = cv2.erode(mask, kernel, iterations=1)
    mask = cv2.dilate(mask, kernel, iterations=2)

    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    state = "SAFE"
    color = (0, 255, 0)

    if len(contours) > 0:
        max_contour = max(contours, key=cv2.contourArea)

        if cv2.contourArea(max_contour) > 1000:
            cv2.drawContours(frame, [max_contour], -1, (0, 255, 255), 2)

            topmost = tuple(max_contour[max_contour[:,:,1].argmin()][0])
            finger_x, finger_y = topmost

            cv2.circle(frame, (finger_x, finger_y), 8, (255, 0, 255), -1)

            distance = math.sqrt((finger_x - center_x)**2 + (finger_y - center_y)**2)

            if distance < DANGER_DIST:
                state = "DANGER"
                color = (0, 0, 255)
            elif distance < SAFE_DIST:
                state = "WARNING"
                color = (0, 165, 255)
            else:
                state = "SAFE"
                color = (0, 255, 0)

            cv2.line(frame, (finger_x, finger_y), (center_x, center_y), color, 2)

    thickness = -1 if state == "DANGER" else 2
    cv2.circle(frame, (center_x, center_y), OBJECT_RADIUS, color, thickness)

    cv2.putText(frame, f"STATUS: {state}", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

    if state == "DANGER":
        cv2.putText(frame, "DANGER DANGER", (center_x - 150, center_y - 80), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 3)

    cv2.imshow("Mask Helper", cv2.resize(mask, (300, 200)))
    cv2.imshow(window_name, frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()